# Example 03 - Official WidebandSig53 Dataset
This notebook walks through how to use `torchsig` to generate the Official WidebandSig53 Dataset.

-------------------------------------------

### Import Libraries

In [1]:
from torchsig.utils.visualize import (
    MaskClassVisualizer,
    mask_class_to_outline,
    complex_spectrogram_to_magnitude,
)
from torchsig.transforms.target_transforms import DescToMaskClass, DescToListTuple
from torchsig.transforms import Spectrogram, Normalize
from torchsig.utils.writer import DatasetCreator, DatasetLoader
from torchsig.datasets.wideband_sig53 import WidebandSig53
from torchsig.datasets.wideband import WidebandModulationsDataset
from torchsig.transforms.transforms import Compose
from torchsig.datasets import conf
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

-----------------------------
## Generate the Wideband Sig53 Dataset
To generate the WidebandSig53 dataset, several parameters are given to the imported `WidebandSig53` class. These paramters are:
- `root` ~ A string to specify the root directory of where to generate and/or read an existing WidebandSig53 dataset
- `train` ~ A boolean to specify if the WidebandSig53 dataset should be the training (True) or validation (False) sets
- `impaired` ~ A boolean to specify if the WidebandSig53 dataset should be the clean version or the impaired version
- `transform` ~ Optionally, pass in any data transforms here if the dataset will be used in an ML training pipeline. Note: these transforms are not called during the dataset generation. The static saved dataset will always be in IQ format. The transform is only called when retrieving data examples.
- `target_transform` ~ Optionally, pass in any target transforms here if the dataset will be used in an ML training pipeline. Note: these target transforms are not called during the dataset generation. The static saved dataset will always be saved as tuples in the LMDB dataset. The target transform is only called when retrieving data examples.
- `use_signal_data` ~ Optionally, pass in a boolean to specify if the annotations should be interpreted as `SignalData` objects as LMDB data is read. This is necessary when using the TorchSig pipeline; however, setting the value to False will simply return the annotations as a list of tuples as it is saved in the LMDB static data

A combination of the `train` and the `impaired` booleans determines which of the four (4) distinct WidebandSig53 datasets will be instantiated:
- `train=True` & `impaired=False` = Clean training set of 250k examples
- `train=True` & `impaired=True` = Impaired training set of 250k examples
- `train=False` & `impaired=False` = Clean validation set of 25k examples
- `train=False` & `impaired=True` = Impaired validation set of 25k examples

The final option of the impaired validation set is the dataset to be used when reporting any results with the official WidebandSig53 dataset.

In [2]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [4]:
cfg = conf.WidebandSig53ImpairedTrainQAConfig
# cfg = conf.WidebandSig53CleanTrainConfig

wideband_ds = WidebandModulationsDataset(
    level=cfg.level,
    num_iq_samples=cfg.num_iq_samples,
    num_samples=cfg.num_samples,
    target_transform=DescToListTuple(),
    seed=cfg.seed,
)

dataset_loader = DatasetLoader(wideband_ds, batch_size=8, num_workers=8, seed=12345678, collate_fn=collate_fn)
creator = DatasetCreator(
    wideband_ds,
    seed=12345678,
    path="/project/def-msteve/torchsig/sig53_wideband/wideband_sig53_impaired_train",
    loader=dataset_loader,
)
creator.create()

MemoryError: /project/def-msteve/torchsig/sig53_wideband/wideband_sig53_impaired_train: Cannot allocate memory

In [ ]:
# cfg = conf.WidebandSig53CleanValQAConfig
cfg = conf.WidebandSig53ImpairedValQAConfig

wideband_ds = WidebandModulationsDataset(
    level=cfg.level,
    num_iq_samples=cfg.num_iq_samples,
    num_samples=cfg.num_samples,
    target_transform=DescToListTuple(),
    seed=cfg.seed,
)

dataset_loader = DatasetLoader(wideband_ds, batch_size=8, num_workers=8, seed=12345678, collate_fn=collate_fn)
creator = DatasetCreator(
    wideband_ds,
    seed=12345678,
    path="/project/def-msteve/torchsig/sig53_wideband/wideband_sig53_impaired_val",
    loader=dataset_loader,
)
creator.create()

In [ ]:
# Specify WidebandSig53 Options
root = "/project/def-msteve/torchsig/sig53_wideband/"
train = False
impaired = True
fft_size = 512
num_classes = 53

transform = Compose(
    [
        Spectrogram(nperseg=fft_size, noverlap=0, nfft=fft_size, mode="complex"),
        Normalize(norm=np.inf, flatten=True),
    ]
)

target_transform = Compose(
    [
        DescToMaskClass(num_classes=num_classes, width=fft_size, height=fft_size),
    ]
)

# Instantiate the WidebandSig53 Dataset
wideband_sig53 = WidebandSig53(
    root=root,
    train=train,
    impaired=impaired,
    transform=transform,
    target_transform=target_transform,
    use_signal_data=True,
)

# Retrieve a sample and print out information
idx = np.random.randint(len(wideband_sig53))
data, label = wideband_sig53[idx]
print("Dataset length: {}".format(len(wideband_sig53)))
print("Data shape: {}".format(data.shape))
print("Label shape: {}".format(label.shape))

### Plot Subset to Verify
The `MaskClassVisualizer` can be passed a `Dataloader` and plot visualizations of the dataset. The `batch_size` of the `DataLoader` determines how many examples to plot for each iteration over the visualizer. Note that the dataset itself can be indexed and plotted sequentially using any familiar python plotting tools as an alternative plotting method to using the `spdata` `Visualizer` as shown below.

In [ ]:
data_loader = DataLoader(
    dataset=wideband_sig53,
    batch_size=16,
    shuffle=True,
)

visualizer = MaskClassVisualizer(
    data_loader=data_loader,
    visualize_transform=complex_spectrogram_to_magnitude,
    visualize_target_transform=mask_class_to_outline,
    class_list=wideband_sig53.modulation_list,
)

for figure in iter(visualizer):
    figure.set_size_inches(16, 9)
    plt.show()
    break

----
### Analyze Dataset
The dataset can also be analyzed at the macro level for details such as the distribution of classes and number of signals per sample. The below analysis reads information directly from the non-target transformed tuple annotations. Since this is different than the above dataset instantiation, the dataset is re-instantiated for analysis.

In [ ]:
# Re-instantiate the WidebandSig53 Dataset without a target transform and without using the RFData objects
wideband_sig53 = WidebandSig53(
    root=root,
    train=train,
    impaired=impaired,
    transform=transform,
    target_transform=None,
)

# Loop through the dataset recording classes and SNRs
class_counter_dict = {
    class_name: 0 for class_name in list(wideband_sig53.modulation_list)
}
num_signals_per_sample = []

for idx in tqdm(range(len(wideband_sig53))):
    data, annotation = wideband_sig53[idx]
    num_signals_per_sample.append(len(annotation))
    for signal_annotation in annotation:
        class_counter_dict[signal_annotation.class_name] += 1

In [ ]:
# Plot the distribution of classes
class_names = list(class_counter_dict.keys())
num_classes = list(class_counter_dict.values())

plt.figure(figsize=(9, 9))
plt.pie(num_classes, labels=class_names)
plt.title("Class Distribution Pie Chart")
plt.show()

plt.figure(figsize=(11, 4))
plt.bar(class_names, num_classes)
plt.xticks(rotation=90)
plt.title("Class Distribution Bar Chart")
plt.xlabel("Modulation Class Name")
plt.ylabel("Counts")
plt.show()

The above distribution of classes shows all OFDM signals appearing less frequently than the remaining modulations. This makes sense because OFDM signals are drawn from a random distribution of bandwidths that are inherently larger than the remaining signals, meaning fewer OFDM signals can fit into a wideband spectrum without overlapping. Additionally, the random bursty probability and durations of OFDM signals makes it less likely to occupy a wideband capture with many short-time bursts, while the remaining modulations experience this behavior at a higher probility.

In [ ]:
# Plot the distribution of number of signals per sample
plt.figure(figsize=(11, 8))
plt.hist(
    x=num_signals_per_sample, bins=np.arange(1, max(num_signals_per_sample) + 1) - 0.5
)
plt.title(
    "Distribution of Number of Signals Per Sample\nTotal Number: {} - Average: {} - Max: {}".format(
        sum(num_signals_per_sample),
        np.mean(np.asarray(num_signals_per_sample)),
        max(num_signals_per_sample),
    )
)
plt.xlabel("Number of Signal Bins")
plt.ylabel("Counts")
plt.show()

The above distribution of the number of signals per sample shows the most commonly seen sample has two signals present. The average is slightly around 4 signals per sample and the max is 26.

In [ ]:
# For additional analysis, reinstantiate the dataset without a target transform, such that the RFDescriptions can be read
wideband_sig53 = WidebandSig53(
    root=root,
    train=train,
    impaired=impaired,
    transform=None,
    target_transform=None,
)

num_samples = len(wideband_sig53)
snrs = []
bandwidths = []
for idx in tqdm(range(num_samples)):
    label = wideband_sig53[idx][1]
    for rf_desc in label:
        snrs.append(rf_desc.snr)
        bandwidths.append(rf_desc.bandwidth)

In [ ]:
# Plot the distribution of SNR values
plt.figure(figsize=(11, 4))
plt.hist(x=snrs, bins=100)
plt.title("SNR Distribution")
plt.xlabel("SNR Bins (dB)")
plt.ylabel("Counts")
plt.show()

In [ ]:
# Plot the distribution of bandwidth values
plt.figure(figsize=(11, 4))
plt.hist(x=bandwidths, bins=100)
plt.title("Bandwidth Distribution")
plt.xlabel("BW Bins")
plt.ylabel("Counts")
plt.show()